# 鐵達尼號生還預測：ROC曲線分析（修正版）

本筆記本專門處理原始資料中的缺失值問題：
- Cabin: 大量缺失值
- Age: 部分缺失值
- Embarked: 少量缺失值
- Fare: 測試資料中可能有缺失值

## 1. 環境準備

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, 
    roc_curve, auc, roc_auc_score, f1_score, precision_score, 
    recall_score, precision_recall_curve
)

# 設定中文字體和圖表樣式
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("環境準備完成！")

## 2. 資料預處理函式（針對缺失值優化）

In [ ]:
def analyze_missing_values(df, dataset_name="資料集"):
    """
    分析資料集中的缺失值情況
    """
    print(f"\n=== {dataset_name} 缺失值分析 ===")
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df)) * 100
    
    missing_df = pd.DataFrame({
        '缺失數量': missing_count,
        '缺失百分比': missing_percent
    })
    
    # 只顯示有缺失值的欄位
    missing_df = missing_df[missing_df['缺失數量'] > 0].sort_values('缺失數量', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("沒有發現缺失值")
    
    return missing_df


def preprocess_titanic_data(df, dataset_name="資料集"):
    """
    對Titanic資料進行完整的預處理，重點處理缺失值
    """
    print(f"\n開始處理 {dataset_name}...")
    data = df.copy()
    
    # 1. 分析原始缺失值
    analyze_missing_values(data, f"{dataset_name}（原始）")
    
    # 2. 處理缺失值
    print(f"\n處理 {dataset_name} 的缺失值...")
    
    # Age: 用中位數填補
    if data['Age'].isnull().any():
        age_median = data['Age'].median()
        age_missing_count = data['Age'].isnull().sum()
        data['Age'].fillna(age_median, inplace=True)
        print(f"✓ Age: {age_missing_count} 個缺失值用中位數 {age_median:.1f} 填補")
    
    # Embarked: 用眾數填補
    if data['Embarked'].isnull().any():
        embarked_mode = data['Embarked'].mode()[0]
        embarked_missing_count = data['Embarked'].isnull().sum()
        data['Embarked'].fillna(embarked_mode, inplace=True)
        print(f"✓ Embarked: {embarked_missing_count} 個缺失值用眾數 '{embarked_mode}' 填補")
    
    # Fare: 用中位數填補（主要針對測試資料）
    if data['Fare'].isnull().any():
        fare_median = data['Fare'].median()
        fare_missing_count = data['Fare'].isnull().sum()
        data['Fare'].fillna(fare_median, inplace=True)
        print(f"✓ Fare: {fare_missing_count} 個缺失值用中位數 {fare_median:.2f} 填補")
    
    # 3. 特徵工程
    print(f"\n進行 {dataset_name} 的特徵工程...")
    
    # 提取稱謂
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    title_mapping = {
        'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
        'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare',
        'Mlle': 'Miss', 'Countess': 'Rare', 'Ms': 'Miss', 'Lady': 'Rare',
        'Jonkheer': 'Rare', 'Don': 'Rare', 'Dona': 'Rare', 'Mme': 'Mrs',
        'Capt': 'Rare', 'Sir': 'Rare'
    }
    data['Title'] = data['Title'].map(title_mapping)
    data['Title'].fillna('Rare', inplace=True)
    
    # 家庭相關特徵
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    
    # Cabin特徵：將缺失值轉換為有用的特徵
    data['Has_Cabin'] = (~data['Cabin'].isnull()).astype(int)
    cabin_missing_count = data['Cabin'].isnull().sum()
    print(f"✓ Cabin: {cabin_missing_count} 個缺失值轉換為 Has_Cabin 特徵")
    
    # 票價對數轉換
    data['Fare_log'] = np.log(data['Fare'] + 1)
    
    # 年齡分組
    def categorize_age(age):
        if age <= 11:
            return 'Child(11以下)'
        elif age <= 18:
            return 'Teen(12-18)'
        elif age <= 58:
            return 'Adult(19-58)'
        else:
            return 'Senior(59以上)'
    
    data['Age_Group'] = data['Age'].apply(categorize_age)
    
    # 4. 最終檢查
    analyze_missing_values(data, f"{dataset_name}（處理後）")
    
    return data

print("資料預處理函式定義完成！")

## 3. 載入並預處理資料

In [ ]:
# 定義資料路徑
train_path = r'C:\Users\jedi8\Documents\GitHub\IanLi-Data-Analytics-Projects\projects\01_Exploratory_Data_Analysis\001_EDA_Project_A_Titanic_Survival_Analysis\data\train.csv'
test_path = r'C:\Users\jedi8\Documents\GitHub\IanLi-Data-Analytics-Projects\projects\01_Exploratory_Data_Analysis\001_EDA_Project_A_Titanic_Survival_Analysis\data\test.csv'

# 載入原始資料
print("載入原始資料...")
train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

print(f"訓練資料: {train_raw.shape[0]} 筆, {train_raw.shape[1]} 欄")
print(f"測試資料: {test_raw.shape[0]} 筆, {test_raw.shape[1]} 欄")

# 預處理資料
train_processed = preprocess_titanic_data(train_raw, "訓練資料")
test_processed = preprocess_titanic_data(test_raw, "測試資料")

## 4. 準備模型訓練資料

In [ ]:
# 定義要移除的欄位
features_to_drop = ['PassengerId', 'Name', 'Ticket', 'SibSp', 'Parch', 'Fare', 'Age', 'Cabin']
target_column = 'Survived'

# 準備訓練資料
print("\n準備訓練資料...")
categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'Age_Group']
train_encoded = pd.get_dummies(train_processed, columns=categorical_features, drop_first=True)

y_train = train_encoded[target_column]
X_train = train_encoded.drop(columns=[target_column] + features_to_drop, axis=1)
X_train = X_train.apply(pd.to_numeric)

# 記錄特徵欄位
feature_columns = X_train.columns.tolist()

print(f"訓練特徵數量: {len(feature_columns)}")
print(f"訓練樣本數: {len(X_train)}")
print(f"\n使用的特徵:")
for i, col in enumerate(feature_columns, 1):
    print(f"{i:2d}. {col}")

## 5. 準備測試資料

In [ ]:
# 準備測試資料
print("\n準備測試資料...")
test_encoded = pd.get_dummies(test_processed, columns=categorical_features, drop_first=True)

# 檢查是否有Survived欄位
has_target = 'Survived' in test_encoded.columns
print(f"測試資料是否包含答案: {'是' if has_target else '否'}")

# 移除不需要的欄位
columns_to_drop = features_to_drop.copy()
if has_target:
    columns_to_drop.append('Survived')

X_test = test_encoded.drop(columns=[col for col in columns_to_drop if col in test_encoded.columns], axis=1)

# 確保特徵一致性
print("\n檢查特徵一致性...")
missing_features = [col for col in feature_columns if col not in X_test.columns]
extra_features = [col for col in X_test.columns if col not in feature_columns]

if missing_features:
    print(f"添加缺失特徵: {missing_features}")
    for col in missing_features:
        X_test[col] = 0

if extra_features:
    print(f"移除多餘特徵: {extra_features}")

# 重新排序特徵
X_test = X_test[feature_columns]
X_test = X_test.apply(pd.to_numeric)

# 提取目標變數（如果存在）
y_test = test_encoded['Survived'] if has_target else None

print(f"測試樣本數: {len(X_test)}")
print(f"特徵匹配: ✓")

## 6. 模型訓練

In [ ]:
# 訓練羅吉斯迴歸模型
print("\n訓練羅吉斯迴歸模型...")
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

# 訓練集預測
y_train_pred = log_reg.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

print(f"模型訓練完成！")
print(f"訓練集準確率: {train_accuracy:.4f}")

## 7. 測試資料預測與ROC分析

In [ ]:
# 對測試資料進行預測
y_pred_test = log_reg.predict(X_test)
y_pred_proba_test = log_reg.predict_proba(X_test)[:, 1]

print(f"\n=== 測試資料預測結果 ===")
print(f"預測生還: {np.sum(y_pred_test)} 人 ({np.mean(y_pred_test)*100:.1f}%)")
print(f"預測未生還: {len(y_pred_test) - np.sum(y_pred_test)} 人 ({(1-np.mean(y_pred_test))*100:.1f}%)")
print(f"平均預測機率: {np.mean(y_pred_proba_test):.4f}")

# 預測結果視覺化
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 預測機率分布
axes[0].hist(y_pred_proba_test, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_xlabel('預測生還機率')
axes[0].set_ylabel('乘客數量')
axes[0].set_title('預測機率分布')
axes[0].grid(True, alpha=0.3)

# 預測結果分布
survival_counts = pd.Series(y_pred_test).value_counts().sort_index()
axes[1].bar(['未生還', '生還'], survival_counts.values, color=['red', 'green'], alpha=0.7)
axes[1].set_ylabel('乘客數量')
axes[1].set_title('預測結果分布')
for i, v in enumerate(survival_counts.values):
    axes[1].text(i, v + 5, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# ROC分析（如果有真實答案）
if y_test is not None:
    print(f"\n=== ROC曲線分析 ===")
    
    # 計算ROC曲線
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_test)
    roc_auc = auc(fpr, tpr)
    
    # 計算其他指標
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    
    print(f"測試集準確率: {test_accuracy:.4f}")
    print(f"測試集精確率: {test_precision:.4f}")
    print(f"測試集召回率: {test_recall:.4f}")
    print(f"測試集F1分數: {test_f1:.4f}")
    print(f"AUC值: {roc_auc:.4f}")
else:
    print(f"\n=== 注意 ===")
    print("測試資料不包含真實答案，無法進行ROC分析")

## 8. ROC曲線繪製（僅當有真實答案時）

In [ ]:
if y_test is not None:
    # 繪製ROC曲線
    plt.figure(figsize=(10, 8))
    
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC曲線 (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
             label='隨機分類器 (AUC = 0.5)')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('偽陽性率 (False Positive Rate)', fontsize=12)
    plt.ylabel('真陽性率 (True Positive Rate)', fontsize=12)
    plt.title('羅吉斯迴歸模型 - ROC曲線', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # 添加AUC解釋
    auc_interpretation = ""
    if roc_auc > 0.9:
        auc_interpretation = "優秀"
    elif roc_auc > 0.8:
        auc_interpretation = "良好"
    elif roc_auc > 0.7:
        auc_interpretation = "尚可"
    elif roc_auc > 0.6:
        auc_interpretation = "較差"
    else:
        auc_interpretation = "無效"
    
    plt.text(0.6, 0.2, 
             f'AUC = {roc_auc:.4f}\n模型效能: {auc_interpretation}\n\n解釋:\n• AUC > 0.9: 優秀\n• AUC > 0.8: 良好\n• AUC > 0.7: 尚可\n• AUC > 0.6: 差\n• AUC ≤ 0.5: 無效', 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7),
             fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # 找到最佳閾值
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    print(f"\n最佳分類閾值: {optimal_threshold:.4f}")
    print(f"在此閾值下，TPR = {tpr[optimal_idx]:.4f}, FPR = {fpr[optimal_idx]:.4f}")
    
else:
    print("\n由於測試資料沒有真實答案，無法繪製ROC曲線")
    print("如需ROC分析，請使用包含Survived欄位的測試資料")

## 9. 保存預測結果

In [ ]:
# 保存預測結果
if 'PassengerId' in test_raw.columns:
    results_df = pd.DataFrame({
        'PassengerId': test_raw['PassengerId'],
        'Survived': y_pred_test,
        'Survival_Probability': y_pred_proba_test
    })
else:
    results_df = pd.DataFrame({
        'Index': range(len(y_pred_test)),
        'Survived': y_pred_test,
        'Survival_Probability': y_pred_proba_test
    })

print("\n=== 預測結果 ===")
print("前10筆預測結果:")
print(results_df.head(10))

# 保存到CSV
output_path = '../data/titanic_predictions_roc_analysis.csv'
results_df.to_csv(output_path, index=False)
print(f"\n預測結果已保存至: {output_path}")

print(f"\n最終統計:")
print(f"總樣本數: {len(results_df)}")
print(f"預測生還: {results_df['Survived'].sum()} 人 ({results_df['Survived'].mean()*100:.1f}%)")
print(f"預測未生還: {len(results_df) - results_df['Survived'].sum()} 人 ({(1-results_df['Survived'].mean())*100:.1f}%)")